# KDX 뉴스 데이터 수집

이 노트북은 KDX(한국데이터거래소) 관련 뉴스 데이터를 수집하는 프로세스를 담고 있습니다.

## 목표
- KDX 관련 뉴스 기사를 자동으로 수집
- Nate, MBN 등 다양한 소스에서 데이터 수집
- 수집한 데이터를 구조화된 형태로 저장

In [1]:
# 필요한 라이브러리 임포트
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from datetime import datetime, timedelta
import json
from dotenv import load_dotenv
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import sys
sys.setrecursionlimit(100000)  # 너무 깊은 구조 대비

os.environ["APP_ENV"] = "DEV"

app_env = os.getenv("APP_ENV", "DEV")

load_dotenv(f".env.{app_env}")


# 프로젝트 루트 경로 추가 (상대 경로 임포트를 위한 설정)
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

# 모듈화된 코드 임포트
from src.data.crawler import NewsCrawler
from src.utils.helpers import ensure_directory

## 1. 출력 디렉토리 설정

In [2]:
# 데이터 저장 경로 설정
RAW_DATA_DIR = os.path.join(project_root, 'data', 'raw')
OUTPUT_FILE = os.path.join(RAW_DATA_DIR, 'news_articles.csv')

# 디렉토리 생성
ensure_directory(RAW_DATA_DIR)

print(f"데이터 저장 경로: {OUTPUT_FILE}")

True

데이터 저장 경로: /Users/koko/llm_pipeline/data/raw/news_articles.csv


## 2. 크롤러 설정

In [ ]:
# 크롤러 설정 정의
crawler_config = {
    'search_keywords': ['KDX', '한국데이터거래소', '데이터거래소', 'AI데이터'],
    'news_sources': ['mk'],
    'max_articles_per_source': 15,
    'date_range_days': 30
}

# 설정 내용 확인
print("크롤러 설정:")
for key, value in crawler_config.items():
    print(f"  - {key}: {value}")

## 3. 뉴스 기사 수집 실행

In [ ]:
# 크롤러 초기화
crawler = NewsCrawler()

# 크롤러에 설정 적용 (설정 파일무시)
# crawler.config = crawler_config

# 크롤링 실행
print("크롤링 시작...")
start_time = time.time()

articles_df = crawler.run(OUTPUT_FILE)

end_time = time.time()
execution_time = end_time - start_time

print(f"크롤링 완료! 소요 시간: {execution_time:.2f}초")
print(f"수집된 기사 수: {len(articles_df)}")

## 4. 수집 결과 확인

In [ ]:
# 데이터프레임 미리보기
articles_df.head()

In [ ]:
# 소스별 기사 수 확인
source_counts = articles_df['source'].value_counts()
print("소스별 기사 수:")
print(source_counts)

In [ ]:
# 소스별 기사 수 시각화
plt.figure(figsize=(10, 6))
source_counts.plot(kind='bar')
plt.title('뉴스 소스별 KDX 관련 기사 수', fontsize=15)
plt.xlabel('뉴스 소스', fontsize=12)
plt.ylabel('기사 수', fontsize=12)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# 기사 내용 샘플 확인
sample_article = articles_df.iloc[0]
print(f"제목: {sample_article['title']}")
print(f"날짜: {sample_article['date']}")
print(f"소스: {sample_article['source']}")
print(f"URL: {sample_article['url']}")
print("\n내용:")
print(sample_article['content'][:500] + '...' if len(sample_article['content']) > 500 else sample_article['content'])

## 5. 데이터 전처리 실행

In [3]:
# 데이터 전처리 모듈 임포트
from src.data.preprocessor import NewsPreprocessor

# 전처리 출력 파일 경로 설정
PROCESSED_DATA_PATH = os.path.join(RAW_DATA_DIR, 'processed_news_articles.csv')

# 전처리기 초기화 및 실행
print("데이터 전처리 시작...")
start_time = time.time()

preprocessor = NewsPreprocessor()
processed_df = preprocessor.run(OUTPUT_FILE, PROCESSED_DATA_PATH)

end_time = time.time()
execution_time = end_time - start_time

print(f"전처리 완료! 소요 시간: {execution_time:.2f}초")
print(f"전처리된 기사 수: {len(processed_df)}")

# 전처리 결과 미리보기
processed_df.head()

main        : INFO     데이터프레임 전처리 시작
main        : INFO     전처리 완료: 61개 기사
main        : INFO     전처리된 데이터 저장 완료: /Users/koko/llm_pipeline/data/raw/processed_news_articles.csv


데이터 전처리 시작...
전처리 완료! 소요 시간: 0.03초
전처리된 기사 수: 61


,title,content,date,url,source,title_clean,content_clean,article_id
0,한우 명인과 고기굽기 장인이 만나면? “투뿔 그 이상의 가치”,21일 오후 4시 서울 서초구 영주축협 한우프라자 청계산역점. 그동안 단 한 번도 ...,2025-04-22,https://www.mk.co.kr/news/economy/11297884,매일경제,한우 명인과 고기굽기 장인이 만나면 투뿔 그 이상의 가치,21일 오후 4시 서울 서초구 영주축협 한우프라자 청계산역점. 그동안 단 한 번도 ...,매일경제_0
1,"KDX한국데이터거래소, AI학습용 데이터 판매",KDX한국데이터거래소(이하 KDX)가 매일경제신문의 ‘2024년 Special Ed...,2025-04-10,https://www.mk.co.kr/news/it/11287762,매일경제,KDX한국데이터거래소 AI학습용 데이터 판매,KDX한국데이터거래소 이하 KDX 가 매일경제신문의 2024년 Special Edi...,매일경제_1
2,[KDX한국데이터거래소] 3월 주목할 데이터,"▲편의점의 모든 순간, 소비 트렌드를 담다\n2020년 이후 주요 유통사들의 점포 ...",2025-04-03,https://www.mk.co.kr/news/it/11281722,매일경제,KDX한국데이터거래소 3월 주목할 데이터,편의점의 모든 순간 소비 트렌드를 담다 2020년 이후 주요 유통사들의 점포 확장 ...,매일경제_2
3,"성균관대, ‘2025 국제 컨퍼런스 시상식’ 성료",성균관대학교는 지난 26일(수) 수원 자연과학캠퍼스 산학협력센터 85447에서 ‘2...,2025-03-27,https://www.mk.co.kr/news/it/11275730,매일경제,성균관대 2025 국제 컨퍼런스 시상식 성료,성균관대학교는 지난 26일 수 수원 자연과학캠퍼스 산학협력센터 85447에서 202...,매일경제_3
4,매경·재능대 ‘농식품AI아카데미’ 제6기 돌입,매일경제가 재능대학교와 공동 운영하는 ‘농식품 인공지능(AI) 아카데미’가 제6기 ...,2025-03-19,https://www.mk.co.kr/news/economy/11267501,매일경제,매경 재능대 농식품AI아카데미 제6기 돌입,매일경제가 재능대학교와 공동 운영하는 농식품 인공지능 AI 아카데미 가 제6기 과정...,매일경제_4


## 6. 요약 생성

In [4]:
# 요약 모듈 임포트
from src.models.summarizer import GPTSummarizer

# 요약 출력 파일 경로 설정
SUMMARY_DATA_PATH = os.path.join(RAW_DATA_DIR, 'news_summaries.csv')

# 요약 생성기 초기화 및 실행
print("뉴스 요약 생성 시작...")
start_time = time.time()

summarizer = GPTSummarizer()
summary_df = summarizer.run(PROCESSED_DATA_PATH, SUMMARY_DATA_PATH)

end_time = time.time()
execution_time = end_time - start_time

print(f"요약 생성 완료! 소요 시간: {execution_time:.2f}초")
print(f"요약된 기사 수: {len(summary_df)}")

# 요약 결과 미리보기
summary_df[['title', 'summary']].head()

main        : INFO     진행 중: 0/61 기사 처리 완료


뉴스 요약 생성 시작...


main        : ERROR    요약 생성 중 오류 발생 (시도 1/3): Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
main        : INFO     1초 후 재시도...
main        : ERROR    요약 생성 중 오류 발생 (시도 2/3): Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}
main        : INFO     2초 후 재시도...
main        : ERROR    요약 생성 중 오류 발생 (시도 3/3): Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs

KeyboardInterrupt: 

## 7. 시각화 생성

In [ ]:
# 시각화 모듈 임포트
from src.visualization.plotter import NewsVisualizer

# 시각화 출력 디렉토리 설정
VIZ_OUTPUT_DIR = os.path.join(project_root, 'data', 'visualizations')
ensure_directory(VIZ_OUTPUT_DIR)

# 시각화 생성기 초기화 및 실행
print("시각화 생성 시작...")
start_time = time.time()

visualizer = NewsVisualizer()
viz_results = visualizer.run(SUMMARY_DATA_PATH, VIZ_OUTPUT_DIR)

end_time = time.time()
execution_time = end_time - start_time

print(f"시각화 생성 완료! 소요 시간: {execution_time:.2f}초")
print(f"생성된 시각화 파일: {viz_results}")

## 8. 주요 시각화 결과 확인

In [ ]:
# 워드 클라우드 표시
wordcloud_path = os.path.join(VIZ_OUTPUT_DIR, 'wordcloud.png')
if os.path.exists(wordcloud_path):
    from IPython.display import Image
    display(Image(filename=wordcloud_path))
else:
    print("워드 클라우드 이미지를 찾을 수 없습니다.")

In [ ]:
# 시간별 기사 분포
timeline_path = os.path.join(VIZ_OUTPUT_DIR, 'article_timeline.png')
if os.path.exists(timeline_path):
    from IPython.display import Image
    display(Image(filename=timeline_path))
else:
    print("시간별 기사 분포 이미지를 찾을 수 없습니다.")

In [ ]:
# 키워드 네트워크 그래프
network_path = os.path.join(VIZ_OUTPUT_DIR, 'keyword_network.png')
if os.path.exists(network_path):
    from IPython.display import Image
    display(Image(filename=network_path))
else:
    print("키워드 네트워크 그래프 이미지를 찾을 수 없습니다.")

## 9. 요약 및 다음 단계

- KDX 관련 뉴스 데이터를 수집, 전처리, 요약, 시각화하는 전체 파이프라인을 실행했습니다.
- 총 수집된 기사 수: {len(articles_df)}개
- 전처리 및 요약된 기사 수: {len(summary_df)}개
- 주요 키워드: AI, 데이터, 거래소, 플랫폼, 비즈니스 등

### 다음 단계
1. 뉴스 소스 확장 (더 많은 언론사 포함)
2. 감성 분석 추가 (기사의 긍정/부정 분석)
3. 실시간 모니터링 시스템 구축
4. 키워드 트렌드 분석 및 예측 모델 개발

In [ ]:
# 파이프라인 실행 요약 정보 저장
summary_info = {
    "execution_timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "articles_collected": len(articles_df),
    "articles_processed": len(processed_df),
    "articles_summarized": len(summary_df),
    "output_files": {
        "raw_data": OUTPUT_FILE,
        "processed_data": PROCESSED_DATA_PATH,
        "summary_data": SUMMARY_DATA_PATH,
        "visualizations": VIZ_OUTPUT_DIR
    }
}

# JSON 형태로 저장
summary_path = os.path.join(RAW_DATA_DIR, 'pipeline_summary.json')
with open(summary_path, 'w', encoding='utf-8') as f:
    json.dump(summary_info, f, ensure_ascii=False, indent=4)

print(f"파이프라인 실행 요약 정보가 {summary_path}에 저장되었습니다.")